### TRAINING MODELS

In [2]:
# IMPORT EVERYTHING
from Architectures.AE_ADJ_v2 import AE

from Data.load import load, load_middle_slices
from Visualizations.latent_space_projections import plot_multiple_datasets
from Visualizations.plots import plot_middle_slices_in_range, plot_models_training_time, compare_models_loss_history, plot_images, compare_models_reconstruction
from Metrics.metrics_tf import MSE_loss, NMSE_loss, NRMSE_loss, SSIM_loss
from Metrics.metrics import NMSE, SSIM, NRMSE, MSE

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, TSNE
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import layers, Model, activations, regularizers
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from skimage.metrics import structural_similarity as ssim

import umap
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pandas as pd
import seaborn as sns
import glob
import tensorflow as tf
import time
import math
import nibabel as nib
import json
from tqdm import tqdm

c:\Users\kiran\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
groups = {}
#groups3D = {}
for group in tqdm(["CN","MCI","AD"],"Opening faster"):
    groups[group]=np.load(open(f"Data/D2-{group}.npy","rb"))
    #groups3D[group]=np.load(open(f"Data/D3-{group}.npy","rb"))

Opening faster: 100%|██████████| 3/3 [00:00<00:00, 32.29it/s]


In [4]:
"""
A2 = AE(name="A2", input_shape=groups["CN"].shape, latent_dim=16,
    encoder_layers=[
        (layers.Conv2D, 16, (3,3), {'strides': 2, 'padding': 'same', 'activation': 'leaky_relu'}),
        (layers.Conv2D, 32, (3,3), {'strides': 2, 'padding': 'same', 'activation': 'leaky_relu'}),
    ],
    decoder_layers=[
        (layers.Conv2D, 32, (3,3), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
        (layers.Conv2D, 16, (3,3), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
        (layers.Conv2D, 1, (3,3), {'padding': 'same'}),
    ], VAE_model=False, optimizer="adam", loss_fn="mse").compile_model()
"""

"""
models_list = [
    AE(name="public_repo_", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (2, 2)}),
            (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
            (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (2, 2)}),
            (layers.Conv2D, 128, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
            (layers.Conv2D, 128, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization()
        ],
        decoder_layers=[
            (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
            (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same',}),
        ],
        VAE_model=False, optimizer="adam", loss_fn=MSE_loss).compile_model(),
    
    AE(name="public_alcoholism_paper_2D_ssim", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 40, (5, 5), {'activation': 'relu', 'strides': (3, 3), 'padding': 'valid'}), layers.BatchNormalization(),
            (layers.Conv2D, 80, (3, 3), {'activation': 'relu', 'strides': (5, 5), 'padding': 'valid'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (3, 3), 'strides': (1, 1), 'padding': 'same'}),
            (layers.Conv2D, 120, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (3, 3), 'strides': (1, 1), 'padding': 'same'}),
            (layers.Conv2D, 120, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (3, 3), 'strides': (1, 1), 'padding': 'same'}),
        ],
        decoder_layers=[
            (layers.Conv2D, 120, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 120, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 80, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 80, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 40, (5, 5), {'padding': 'same'}), 
            (layers.Conv2D, 1, (3,3), {'padding': 'same'}),
        ], VAE_model=False, optimizer="adam", loss_fn=MSE_loss).compile_model(),
"""

models_list = [
    AE(name="public_unsupervised_anomaly_2d_mse", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 64, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 128, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 16, (1, 1), {'activation': 'relu', 'strides': 1, 'padding': 'same'}), 
        ],
        decoder_layers=[
            (layers.Conv2D, 16, (1, 1), {'activation': 'relu', 'strides': 1, 'padding': 'same'}), 
            (layers.Conv2D, 128, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 64, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 32, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 1, (1, 1), {'activation': 'sigmoid', 'padding': 'same'})
        ], VAE_model=False, optimizer="adam", loss_fn=MSE_loss).compile_model(),

    AE(name="public_unsupervised_anomaly_2d_ssim", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 64, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 128, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 16, (1, 1), {'activation': 'relu', 'strides': 1, 'padding': 'same'}), 
        ],
        decoder_layers=[
            (layers.Conv2D, 16, (1, 1), {'activation': 'relu', 'strides': 1, 'padding': 'same'}), 
            (layers.Conv2D, 128, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 64, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 32, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 1, (1, 1), {'activation': 'sigmoid', 'padding': 'same'})
        ], VAE_model=False, optimizer="adam", loss_fn=SSIM_loss).compile_model(),

    AE(name="public_unsupervised_anomaly_2d_vae_mse", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 64, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 128, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 16, (1, 1), {'activation': 'relu', 'strides': 1, 'padding': 'same'}), 
        ],
        decoder_layers=[
            (layers.Conv2D, 16, (1, 1), {'activation': 'relu', 'strides': 1, 'padding': 'same'}), 
            (layers.Conv2D, 128, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 64, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 32, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 1, (1, 1), {'activation': 'sigmoid', 'padding': 'same'})
        ], VAE_model=True, optimizer="adam", loss_fn=MSE_loss).compile_model(),

    AE(name="public_unsupervised_anomaly_2d_vae_ssim", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 64, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 128, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 16, (1, 1), {'activation': 'relu', 'strides': 1, 'padding': 'same'}), 
        ],
        decoder_layers=[
            (layers.Conv2D, 16, (1, 1), {'activation': 'relu', 'strides': 1, 'padding': 'same'}), 
            (layers.Conv2D, 128, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 64, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 32, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 1, (1, 1), {'activation': 'sigmoid', 'padding': 'same'})
        ], VAE_model=True, optimizer="adam", loss_fn=SSIM_loss).compile_model(),

    AE(name="public_frontiers_paper_2d_mse", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), (layers.MaxPooling2D, {'pool_size': (2, 2), 'strides': (2, 2), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (2, 2), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), 
        ],
        decoder_layers=[
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same', 'strides': 2}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), 
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2DTranspose, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same'}),
        ], VAE_model=False,  optimizer="adam", loss_fn=MSE_loss).compile_model(),
    
        AE(name="public_frontiers_paper_2d_ssim", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), (layers.MaxPooling2D, {'pool_size': (2, 2), 'strides': (2, 2), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (2, 2), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), 
        ],
        decoder_layers=[
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same', 'strides': 2}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), 
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2DTranspose, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same'}),
        ], VAE_model=False,  optimizer="adam", loss_fn=SSIM_loss).compile_model(),

        AE(name="public_frontiers_paper_2d_vae_mse", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), (layers.MaxPooling2D, {'pool_size': (2, 2), 'strides': (2, 2), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (2, 2), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), 
        ],
        decoder_layers=[
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same', 'strides': 2}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), 
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2DTranspose, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same'}),
        ], VAE_model=True,  optimizer="adam", loss_fn=MSE_loss).compile_model(),

        AE(name="public_frontiers_paper_2d_vae_ssim", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), (layers.MaxPooling2D, {'pool_size': (2, 2), 'strides': (2, 2), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (2, 2), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), 
        ],
        decoder_layers=[
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same', 'strides': 2}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), 
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2DTranspose, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same'}),
        ], VAE_model=True,  optimizer="adam", loss_fn=SSIM_loss).compile_model(),
]

"""

"""

"""
models_list = [
    AE(name="AE (3h, 32-128f, 16L, Batchnorm, SSIM_loss)", input_shape=groups["CN"].shape, latent_dim=16,
        encoder_layers=[
            (layers.Conv2D, 32, (5,5), {'strides': 2, 'padding': 'same', 'activation': 'relu'}), layers.BatchNormalization(),
            (layers.Conv2D, 64, (3,3), {'strides': 2, 'padding': 'same', 'activation': 'relu'}), layers.BatchNormalization(),
            (layers.Conv2D, 128, (3,3), {'strides': 2, 'padding': 'same', 'activation': 'relu'}), layers.BatchNormalization(),
        ],
        decoder_layers=[
            (layers.Conv2D, 128, (3,3), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
            (layers.Conv2D, 64,  (3,3), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
            (layers.Conv2D, 32,  (5,5), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
            (layers.Conv2D, 1,   (3,3), {'padding': 'same'}),
        ],
        VAE_model=True,loss=SSIM_loss
)

custom_simplest_model = AE(name="custom_simplest_model", input_shape=H_train2D.shape, latent_dim=16,
    encoder_layers=[
        (layers.Conv2D, 4, (3,3), {'strides': 2, 'padding': 'same', 'activation': 'leaky_relu'}),
    ],
    decoder_layers=[
        (layers.Conv2D, 4, (3,3), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
        (layers.Conv2D, 1, (3,3), {'padding': 'same'}),
    ], VAE_model=False, loss=MSE_loss)


alcoholism = AE(name="public_alcoholism_paper", input_shape=H_train2D.shape, latent_dim=16,
    encoder_layers=[
        (layers.Conv2D, 40, (5, 5), {'activation': 'relu', 'strides': (3, 3), 'padding': 'valid'}), layers.BatchNormalization(),
        (layers.Conv2D, 80, (3, 3), {'activation': 'relu', 'strides': (5, 5), 'padding': 'valid'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (3, 3), 'strides': (1, 1), 'padding': 'same'}),
        (layers.Conv2D, 120, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (3, 3), 'strides': (1, 1), 'padding': 'same'}),
        (layers.Conv2D, 120, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (3, 3), 'strides': (1, 1), 'padding': 'same'}),
    ],
    decoder_layers=[
        (layers.Conv2D, 120, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 120, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 80, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 80, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 40, (5, 5), {'padding': 'same'}), 
        (layers.Conv2D, 1, (3,3), {'padding': 'same'}),
    ],
    VAE_model=False, loss=MSE_loss, learning_rate=0.001
)


frontiers = AE(name="public_frontiers_paper", input_shape=H_train2D.shape, latent_dim=16,
    encoder_layers=[
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}),
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), (layers.MaxPooling2D, {'pool_size': (2, 2), 'strides': (2, 2), 'padding': 'same'}),
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), 
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (2, 2), 'padding': 'same'}), 
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), 
    ],
    decoder_layers=[
        (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}),
        (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same', 'strides': 2}),
        (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), 
        (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2DTranspose, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same'}),
    ],
    VAE_model=False, loss=MSE_loss, learning_rate=0.001
)

    unsupervised_anomaly = AE(name="public_unsupervised_anomaly", input_shape=H_train2D.shape, latent_dim=128,
    encoder_layers=[
        (layers.Conv2D, 32, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
        (layers.Conv2D, 64, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
        (layers.Conv2D, 128, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
        (layers.Conv2D, 16, (1, 1), {'activation': 'relu', 'strides': 1, 'padding': 'same'}), 
    ],
    decoder_layers=[
        (layers.Conv2D, 16, (1, 1), {'activation': 'relu', 'strides': 1, 'padding': 'same'}), 
        (layers.Conv2D, 128, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 64, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 32, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 1, (1, 1), {'activation': 'sigmoid', 'padding': 'same'})
    ],
    VAE_model=False, loss=MSE_loss, learning_rate=0.001
)
]
"""

Interpreted image shape: (80, 96, 1) Pre-flattened latent shape: (None, 10, 12, 16)
Interpreted image shape: (80, 96, 1) Pre-flattened latent shape: (None, 10, 12, 16)
Interpreted image shape: (80, 96, 1) Pre-flattened latent shape: (None, 10, 12, 16)

Interpreted image shape: (80, 96, 1) Pre-flattened latent shape: (None, 10, 12, 16)
Interpreted image shape: (80, 96, 1) Pre-flattened latent shape: (None, 20, 24, 32)
Interpreted image shape: (80, 96, 1) Pre-flattened latent shape: (None, 20, 24, 32)
Interpreted image shape: (80, 96, 1) Pre-flattened latent shape: (None, 20, 24, 32)
Interpreted image shape: (80, 96, 1) Pre-flattened latent shape: (None, 20, 24, 32)


'\nmodels_list = [\n    AE(name="AE (3h, 32-128f, 16L, Batchnorm, SSIM_loss)", input_shape=groups["CN"].shape, latent_dim=16,\n        encoder_layers=[\n            (layers.Conv2D, 32, (5,5), {\'strides\': 2, \'padding\': \'same\', \'activation\': \'relu\'}), layers.BatchNormalization(),\n            (layers.Conv2D, 64, (3,3), {\'strides\': 2, \'padding\': \'same\', \'activation\': \'relu\'}), layers.BatchNormalization(),\n            (layers.Conv2D, 128, (3,3), {\'strides\': 2, \'padding\': \'same\', \'activation\': \'relu\'}), layers.BatchNormalization(),\n        ],\n        decoder_layers=[\n            (layers.Conv2D, 128, (3,3), {\'padding\': \'same\'}), (layers.UpSampling2D, (2,2)),\n            (layers.Conv2D, 64,  (3,3), {\'padding\': \'same\'}), (layers.UpSampling2D, (2,2)),\n            (layers.Conv2D, 32,  (5,5), {\'padding\': \'same\'}), (layers.UpSampling2D, (2,2)),\n            (layers.Conv2D, 1,   (3,3), {\'padding\': \'same\'}),\n        ],\n        VAE_model=True,loss

In [6]:
for m in models_list: #[A2,A4,A5]
    m.train(groups["CN"], epochs=1000, batch_size=128, verbose=True, 
    save_path=r"C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images", 
    save_interval=100, patience=100)

Training public_unsupervised_anomaly_2d_mse on CPU
Input shape: (988, 80, 96)
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 596ms/step - loss: 0.1011 - val_loss: 0.0589
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 528ms/step - loss: 0.0436 - val_loss: 0.0271
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 515ms/step - loss: 0.0266 - val_loss: 0.0201
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 515ms/step - loss: 0.0218 - val_loss: 0.0176
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 503ms/step - loss: 0.0184 - val_loss: 0.0162
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 513ms/step - loss: 0.0166 - val_loss: 0.0145
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 512ms/step - loss: 0.0158 - val_loss: 0.0142
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 513ms/step - loss: 0.0156 - val_loss: 0.0138
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 511ms/step - loss: 0.0153 - val_loss: 0.0135
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 513ms/step - loss: 0.0149 - val_loss: 0.0134
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 523ms/step - loss: 0.0149 - val_loss: 0.0134
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 512ms/step - loss: 0.0149 - val_loss: 0.0133
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 526ms/step - loss:

In [ ]:
# Test to see if loading works
# models_list = [AE.open(r"C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images\public_repo\epoch_9")]

Models loaded successfully.
